In [1]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

Podział dostępności, eksportu itp na zmienne kategoryczne?

# 1. Load data

In [2]:
data = pd.read_csv('data/full_data_filtered.csv')
data = data.set_index('timestamp')
# data = data.drop(columns=['weekday', 'month'])

# Preprocessing

In [3]:
def visualize():
    import sweetviz as sv
    sweet_report = sv.analyze(data)
    sweet_report.show_html('data.html')

#### Drop columns that have a single value

In [4]:
data = data.drop(columns=[
    'Biomass 200 available',
    'Gas 200 available',
    'HPS 100 available',
    'HPS 200 available',
])

#### Transform highly skewed data into categorical

In [5]:
for highly_skew_series, bins in [('Hard coal 100 available', 2),
                                 ('Lignite 500 available', 4),
                                 ('Lignite 1000 available', 4),
                                 ('CEPS_IMP_lag_24', 2),
                                 ('CEPS_IMP_lag_168', 2),
                                 ('SEPS_IMP_lag_24', 2),
                                 ('SEPS_IMP_lag_168', 2),
                                 ('50HzT_EXP_lag_24', 2),
                                 ('50HzT_EXP_lag_168', 2),
                                 ('SVK_EXP_lag_24', 2),
                                 ('SVK_EXP_lag_168', 2),
                                 ('SVK_IMP_lag_24', 3),
                                 ('SVK_IMP_lag_168', 3),
                                 ('LIT_EXP_lag_24', 2),
                                 ('LIT_EXP_lag_168', 2),
                                 ('LIT_IMP_lag_24', 3),
                                 ('LIT_IMP_lag_168', 3),
                                 ('Biomass 200 generation_lag_24', 2),
                                 ('Biomass 200 generation_lag_168', 2),
                                 ('HPS 100 generation_lag_24', 2),
                                 ('HPS 100 generation_lag_168', 2),
                                 ('HPS 200 generation_lag_24', 2),
                                 ('HPS 200 generation_lag_168', 2),
                                 ]:
    data[highly_skew_series] = pd.cut(data[highly_skew_series], bins=bins, labels=list(range(bins)))

In [6]:
def get_benchmark():
    external_predictions = pd.read_csv('data/external_predictions.csv')
    external_predictions.rename(columns={'Unnamed: 0': 'timestamp'}, inplace=True)
    external_predictions.set_index('timestamp', inplace=True)
    real_vs_external_predictions = external_predictions.merge(data['price'], left_index=True, right_index=True)
    rmse_benchmark = mean_squared_error(real_vs_external_predictions['price'], real_vs_external_predictions['forecast_PLN'], squared=False)
    return rmse_benchmark

# Split

In [7]:
X = data.drop(columns=['price'])
y = data[['price']]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=180*24,
    shuffle=False
)

In [8]:
cat_columns = ['weekday', 'month']
num_columns = list(set(X.columns).difference(set(cat_columns)))
num_columns.sort()
# num_columns

# Get pipeline

In [9]:
def get_coefficient_importance(pipeline):
    coefficients = pipeline.named_steps['regression'].coef_
    columns = X.columns
    coefficients_importance = list(zip(columns, coefficients))
    coefficients_importance.sort(key=lambda x: x[1], reverse=True)
    return coefficients_importance

def get_model_result(model):
    transformer_numerical = Pipeline(steps=[('num_trans', StandardScaler())])
    transformer_categorical = Pipeline(steps=[('cat_trans', OneHotEncoder())])
    preprocessor = ColumnTransformer(transformers=[
        ('numerical', transformer_numerical, num_columns),
        ('categorical', transformer_categorical, cat_columns),
    ])
    pipe = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regression', model),
    ])
    pipe.fit(X_train, y_train)
    rmse_result = mean_squared_error(y_test['price'], pipe.predict(X_test), squared=False)
    return rmse_result

In [10]:
def check_all_models():
    print(f'BENCHMARK: {round(get_benchmark(), 2)}\n')

    results = []

    linear_regression_models = [LinearRegression()]
    ridge_models = [Ridge(alpha=a, max_iter=10000)
                    for a in [0.001, 0.01, 0.1, 1, 2, 5, 10, 25, 50, 100, 250, 1000]]
    lasso_models = [Lasso(alpha=a, max_iter=10000)
                    for a in [0.001, 0.01, 0.1, 1, 2, 5, 10, 25, 50, 100, 250, 1000]]
    elastic_net_models = [ElasticNet(alpha=a, l1_ratio=l, max_iter=10000)
                          for a in [0.001, 0.01, 0.1, 1, 2, 5, 10, 25, 50, 100, 250, 1000]
                          for l in [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99]]

    models = linear_regression_models + ridge_models + lasso_models + elastic_net_models

    for model_ in models:
        results.append((model_, round(get_model_result(model_), 2)))

    results.sort(key=lambda x: x[1])
    print(results)

# Best without transformations: 37.68
# Best with categorical transformation: 37.15

# check_all_models()

In [11]:
best_results = []
best_models = [Lasso(alpha=2, max_iter=10000),
               Lasso(alpha=1, max_iter=10000),
               Lasso(alpha=0.1, max_iter=10000),
               ElasticNet(alpha=0.1, l1_ratio=0.9, max_iter=10000),
               ElasticNet(alpha=0.1, l1_ratio=0.8, max_iter=10000),
               ElasticNet(alpha=0.1, l1_ratio=0.7, max_iter=10000),
               ElasticNet(alpha=1, l1_ratio=0.9, max_iter=10000),
               ElasticNet(alpha=0.1, l1_ratio=0.6, max_iter=10000),
               Ridge(alpha=250, max_iter=10000),
               ElasticNet(alpha=0.01, l1_ratio=0.01, max_iter=10000),
               ElasticNet(alpha=0.01, l1_ratio=0.1, max_iter=10000),
               ElasticNet(alpha=0.01, l1_ratio=0.2, max_iter=10000),
               ElasticNet(alpha=0.1, max_iter=10000),
               Ridge(alpha=100, max_iter=10000),
               Ridge(alpha=10, max_iter=10000),
               Ridge(alpha=1, max_iter=10000),
               ]

for model_ in best_models:
    best_results.append((model_, round(get_model_result(model_), 2)))

best_results.sort(key=lambda x: x[1])
best_results

[(Lasso(alpha=1, max_iter=10000), 37.15),
 (Lasso(alpha=0.1, max_iter=10000), 37.34),
 (ElasticNet(alpha=0.1, l1_ratio=0.9, max_iter=10000), 37.36),
 (ElasticNet(alpha=0.1, l1_ratio=0.8, max_iter=10000), 37.48),
 (ElasticNet(alpha=0.1, l1_ratio=0.7, max_iter=10000), 37.59),
 (Lasso(alpha=2, max_iter=10000), 37.63),
 (ElasticNet(alpha=0.1, l1_ratio=0.6, max_iter=10000), 37.7),
 (ElasticNet(alpha=0.1, max_iter=10000), 37.83),
 (ElasticNet(alpha=1, l1_ratio=0.9, max_iter=10000), 37.94),
 (Ridge(alpha=250, max_iter=10000), 38.47),
 (ElasticNet(alpha=0.01, l1_ratio=0.01, max_iter=10000), 38.55),
 (ElasticNet(alpha=0.01, l1_ratio=0.1, max_iter=10000), 38.59),
 (ElasticNet(alpha=0.01, l1_ratio=0.2, max_iter=10000), 38.65),
 (Ridge(alpha=100, max_iter=10000), 39.04),
 (Ridge(alpha=10, max_iter=10000), 40.3),
 (Ridge(alpha=1, max_iter=10000), 40.64)]

In [12]:
best_model = Lasso(alpha=1, max_iter=10000)
rmse_result = get_model_result(best_model)

In [13]:
best_model.predict(X_test)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 100 is different from 83)